In [1]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la
import matplotlib.animation as animation
from IPython.display import HTML
import autograd
import math

In [2]:
class TestFunction:
    def __init__(self, name: str, domain: np.ndarray, glob_min: np.ndarray,
                 fun: callable, grad: callable = None) -> None:

        """ Инициализация параметров функции

        Args:
            domain (np.ndarray): Область определения функции
            glob_min (np.ndarray): Глобальный минимум
            fun (callable): Функция
        """
        self.name = name
        self.domain = domain
        self.glob_min = glob_min
        self.fun = fun
        self.grad = grad

In [3]:
Sphere = TestFunction(
    name = "Sphere",
    fun = lambda p: p[0]*p[0] + p[1]*p[1],
    grad = lambda p: np.array([2 * p[0], 2 * p[1]]),
    domain = np.array([[-3., -3.], [3., 3.]]),
    glob_min = np.array([0., 0., 0.])
    )

In [4]:
Booth = TestFunction(
    name = "Booth",
    fun = lambda p: (p[0] + 2 * p[1] - 7) ** 2 + (2 * p[0] + p[1] - 5) ** 2,
    grad = lambda p: np.array([10 * p[0] + 8 * p[1] - 34, 10 * p[1] + 8 * p[0] - 38]),
    domain = np.array([[-3., -3.], [7., 7.]]),
    glob_min = np.array([1., 3., 0.])
)

In [5]:
def grad(fun: callable, point: np.ndarray, dt: float = 0.00001) -> np.array:

    """Вычисляет градиент функции в точке

    Args:
        fun (callable): Функция искуственного ландшавта
        point (np.ndarray): Точка (массив параметров)
        dt (float, optional): Изменение аргумента. Defaults to 0.00001.

    Returns:
        np.array: _description_
    """

    dxdt = (fun(point + np.array([dt, 0])) - fun(point)) / dt
    dydt = (fun(point + np.array([0, dt])) - fun(point)) / dt
    return np.array([dxdt, dydt])

In [6]:
def Gradient_Descent(function_info: TestFunction, start_params: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, delta: float = 0.001,
                     method = 'handle') -> np.array:
    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):

        # Вычисляем новое значение параметров
        if method == 'handle':
            params = params - lr * function_info.grad(params)
        elif method == 'numerically':
            params = params - lr * grad(fun, params)
        elif method == 'symbolic':
            params = params - lr * symbolic_grad(params)


        # Логируем результат
        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(path)

In [7]:
def Gradient_Descent_Momentum(function_info: TestFunction, start_params: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, lrm: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:
    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = lrm * v_current - lr * function_info.grad(params)
        elif method == 'numerically':
            v_current = lrm * v_current - lr * grad(fun, params)
        elif method == 'symbolic':
            v_current = lrm * v_current - lr * symbolic_grad(params)

        params = params + v_current

        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(path)

In [8]:
def Gradient_Descent_Momentum_Nesterov(function_info: TestFunction, start_params: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, lrm: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:
    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = lrm * v_current - lr * function_info.grad(params + lrm * v_current)
        elif method == 'numerically':
            v_current = lrm * v_current - lr * grad(fun, params + lrm * v_current)
        elif method == 'symbolic':
            v_current = lrm * v_current - lr * symbolic_grad(params + lrm * v_current)

        params = params + v_current

        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(path)

In [23]:
def Adam(function_info: TestFunction, start_params: np.ndarray,
                     max_iter: int = 64, lr: float = 0.1, lrm: float = 0.1,
                     delta: float = 0.001, method = 'handle') -> np.array:

    fun = function_info.fun
    glob_min = function_info.glob_min

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    symbolic_grad = 0
    if method == 'symbolic':
        symbolic_grad = autograd.grad(fun)

    step = 0
    v_current = np.array([0, 0])
    G_current = np.array([0, 0])
    beta_1, beta_2, eps = 0.9, 0.99, 1e-8
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):
        if method == 'handle':
            v_current = beta_1 * v_current + (1 - beta_1) * function_info.grad(params)
            G_current = beta_2 * G_current + (1 - beta_2) * (function_info.grad(params)) ** 2
        elif method == 'numerically':
            v_current = lrm * v_current - lr * grad(fun, params + lrm * v_current)
        elif method == 'symbolic':
            v_current = lrm * v_current - lr * symbolic_grad(params + lrm * v_current)

        params = params - (lr * v_current) / (np.sqrt(G_current + eps))

        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return np.array(path)

In [9]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

draw_legend = True

def update_lines(num, ax):
    global draw_legend
    line = ax.plot(path[:num, 0], path[:num, 1], path[:num, 2], '-o', c='black', label = 'Градиентный спуск', alpha = 0.7)
    if draw_legend:
        draw_legend = False
        ax.legend(loc="upper left")

    return line

def draw_result(fun: callable, domain: np.array, glob_min: np.array, path: np.array, title: str) -> None:

    """Визуализация градиентного спуска

    Args:
        fun (callable): Исходная функция
        domain (np.array): Границы поиска
        glob_min (np.array): Глобальный минимум функции
        path (np.array): История градиентного спуска
        title (str): Содержание заголовка
    """

    global draw_legend
    draw_legend = True

    fig = plt.figure(figsize = (10, 10))
    ax = plt.axes(projection = '3d')

    print(domain)

    x = np.linspace(domain[0, 0], domain[1, 0], 100)
    y = np.linspace(domain[0, 1], domain[1, 1], 100)

    x_grid, y_grid = np.meshgrid(x, y)
    z_grid = fun(np.array([x_grid, y_grid]))

    ax.plot_surface(x_grid, y_grid, z_grid, cmap = 'plasma', alpha=0.5)
    ax.scatter3D(path[0, 0], path[0, 1], path[0, 2], s=100, c="white", lw=2, ec='black', marker = 'D', label="Начальная точка")
    ax.scatter3D(path[-1, 0], path[-1, 1], path[-1, 2], s=190, c="white", lw=2, ec='black', marker = 'X', label="Найденный минимум")
    ax.scatter3D(glob_min[0], glob_min[1], glob_min[2], s=150, c="red", lw=2, ec='black', marker = 'o', label="Глобальный минимум", alpha = 0.7)

    np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
    print(f"Начальная точка:\t\t\t{path[0]}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Найденный минимум:\t\t\t{path[-1]}")
    print(f"Глобальный минимум:\t\t\t{glob_min}")
    print(f"Кол-во итераций:\t\t\t{len(path)}")
    np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
    print(f"Погрешность найденного решения:\t\t{glob_min - path[-1]}")
    fig.text(0.9, 0.1, s=f"Кол-во итераций: {len(path)}", horizontalalignment="right", fontsize = 12)

    ax.set_title(title, fontsize = 12, fontweight="bold",loc="left")
    ax.legend(loc="upper left")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    num_steps = min(100, len(path))
    ani = animation.FuncAnimation(
            fig, update_lines, num_steps, fargs=(ax,), interval=50)

    return ani

plt.ioff()

In [10]:
path = Gradient_Descent(Sphere, np.array([5.0, 5.0]), max_iter = 100, delta = 0.00001, method = 'handle')
anim = draw_result(Sphere.fun, Sphere.domain, Sphere.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3. -3.]
 [ 3.  3.]]
Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[6.13e-06 6.13e-06 7.52e-11]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			62
Погрешность найденного решения:		[-6.13e-06 -6.13e-06 -7.52e-11]


In [11]:
HTML(anim.to_html5_video())

In [12]:
path = Gradient_Descent(Booth, np.array([5., 5.]), method = 'handle')
anim = draw_result(Booth.fun, Booth.domain, Booth.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [7.00e+00 7.00e+00]]
Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 7.07e-06]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			39
Погрешность найденного решения:		[-8.31e-04 -4.15e-04 -7.07e-06]


In [13]:
HTML(anim.to_html5_video())

In [14]:
path = Gradient_Descent_Momentum(Booth, start_params = np.array([5., 5.]), method = 'symbolic')
anim = draw_result(Booth.fun, Booth.domain, Booth.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [7.00e+00 7.00e+00]]
Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.00e+00 3.00e+00 9.75e-07]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			29
Погрешность найденного решения:		[-6.98e-04 6.98e-04 -9.75e-07]


In [15]:
HTML(anim.to_html5_video())

In [16]:
path = Gradient_Descent_Momentum_Nesterov(Booth, start_params = np.array([5., 5.]), max_iter = 100, method = 'handle')
anim = draw_result(Booth.fun, Booth.domain, Booth.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [7.00e+00 7.00e+00]]
Начальная точка:			[5.00 5.00 164.00]
Найденный минимум:			[1.06e+00 3.06e+00 6.22e-02]
Глобальный минимум:			[1.00e+00 3.00e+00 0.00e+00]
Кол-во итераций:			101
Погрешность найденного решения:		[-5.88e-02 -5.88e-02 -6.22e-02]


In [17]:
HTML(anim.to_html5_video())

In [18]:
path = Gradient_Descent_Momentum_Nesterov(Sphere, start_params = np.array([5., 5.]), max_iter = 100, method = 'handle')
anim = draw_result(Sphere.fun, Sphere.domain, Sphere.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [3.00e+00 3.00e+00]]
Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[5.88e-04 5.88e-04 6.92e-07]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			37
Погрешность найденного решения:		[-5.88e-04 -5.88e-04 -6.92e-07]


In [25]:
HTML(anim.to_html5_video())

In [24]:
path = Adam(Sphere, start_params = np.array([5., 5.]), max_iter = 300, method = 'handle')
anim = draw_result(Sphere.fun, Sphere.domain, Sphere.glob_min, path, "Классический градиентный спуск")
plt.ioff()

[[-3.00e+00 -3.00e+00]
 [3.00e+00 3.00e+00]]
Начальная точка:			[5.00 5.00 50.00]
Найденный минимум:			[-3.78e-05 -3.78e-05 2.86e-09]
Глобальный минимум:			[0.00e+00 0.00e+00 0.00e+00]
Кол-во итераций:			111
Погрешность найденного решения:		[3.78e-05 3.78e-05 -2.86e-09]


In [26]:
HTML(anim.to_html5_video())